<a href="https://colab.research.google.com/github/ppopdesk/BCS-recruitment-task/blob/main/annfromscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Low level implementation of Neural Network with loss function as MSE and optimisation algorithm as Gradient Descent

In [ ]:
import numpy as np

Defining Basic math function we are gonna use for the implementation

In [ ]:
def sigmoid(X):
  return 1/(1+np.exp(-X))
def relu(X):
  return np.maximum(0,X)
def sigmoid_prime(z):
  return sigmoid(z)*(1-sigmoid(z))

Network class contains all the methods for the various process happening in a neural network model : 

1) Layers initialisation

2) Forward Propogation

3) gradient Descent method

4) Updating weights and biases based on one backward propogation step

5) Backprop method

6) Evaluation based on test set


In [ ]:
class Network(object):
  def __init__(self, sizes):
    self.num_layers = len(sizes)
    self.sizes = sizes
    self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
    self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]
  def forward_propogate(self,a,activation=None):
    if activation:
      if activation == 'relu':
        for b, w in zip(self.biases, self.weights):
            a = relu(np.dot(w, a)+b)
        return a
      elif activation == 'sigmoid':
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a
    else:
      for b, w in zip(self.biases, self.weights):
        a = np.dot(w, a)+b
      return a
  def gradient_descent(self, training_data, eta, epochs):
    n = len(training_data)
    for j in range(epochs):
      self.update_one_epoch(training_data, eta)
      print("Epoch {0} complete".format(j))
  def update_one_epoch(self, training_data, eta):
    nabla_b = [np.zeros(b.shape) for b in self.biases]
    nabla_w = [np.zeros(w.shape) for w in self.weights]
    for (x, y) in zip(training_data[0],training_data[1]):
      delta_nabla_b, delta_nabla_w = self.backprop(x, y)
      nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
      nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    self.weights = [w-(eta/len(training_data))*nw for w, nw in zip(self.weights, nabla_w)]
    self.biases = [b-(eta/len(training_data))*nb for b, nb in zip(self.biases, nabla_b)]
  def backprop(self, x, y):
      nabla_b = [np.zeros(b.shape) for b in self.biases]
      nabla_w = [np.zeros(w.shape) for w in self.weights]
      activation = x
      activations = [x] # list to store all the activations, layer by layer
      zs = [] # list to store all the z vectors, layer by layer
      for (b, w) in zip(self.biases, self.weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        delta = self.cost_derivative(activations[-1], y) * \
          sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
          z = zs[-l]
          sp = sigmoid_prime(z)
          delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
          nabla_b[-l] = delta
          nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
      return (nabla_b, nabla_w)
  def evaluate(self, test_data):
    test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in zip(training_data[0],training_data[1])]
    return sum(int(x == y) for (x, y) in test_results)
  def cost_derivative(self, output_activations, y):
    return (output_activations-y)